# Notebook: View Dataset Statistics

This notebook is used to explore a given dataset.

## Packages

In [5]:
import pandas as pd

## Constants

In [12]:
DATASET_PATH = "reviews_dataset/balanced_reviews_sentences.csv"

## Code

In [13]:
df = pd.read_csv(DATASET_PATH)
df

,Unnamed: 0,review_id,restaurant_id,page_index,title,date,author_name,author_location,text,rating,restaurant_name,language_code,detected_language,text_noanonymization,sentence_idx
0,0,867921251,21306626,0,"Ist ganz ok, aber nicht überragend",2022-11-10,Kajtek20011,"Andorra, Spanien","Wir hatten Salat, der war geschmacklich in Ord...",3.0,hans im glück,de,de,"Der Laden ist interessant eingerichtet, mit vi...",4
1,1,875219821,7646488,0,Hans im Glück leider ein Flop,2023-01-15,Gunnic,"Düsseldorf, Deutschland",Leider war es etwas kühl und es roch recht pen...,3.0,hans im glück,de,de,Heute sind wir mit der Familie zum Mittagsnack...,1
2,2,849351269,10685715,0,Tiptop,2022-07-23,christianzW2917NM,"Hamburg, Deutschland","Preislich ist es ""okay"".",4.0,hans im glück,de,de,Essen wir man das erwartet. Stabiler Burger. P...,2
3,3,881137672,11895711,0,Immer gut,2023-03-06,Bernhard H,"Groß-Zimmern, Deutschland",Empfehlenswert und immer gut !,4.0,block house,de,de,Wir sind bereits seit über 40 Jahren treue Bes...,6
4,4,901486491,1520135,0,Go on :),2023-07-08,Hallo A,NaN,Ein großes Lob geht noch an die Ortswahl der <...,5.0,l'osteria,de,de,Alles in einem super! Wir sind mit der Familie...,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,2995,862367069,8121708,1,Für schmächtige reicht eine Pizza völlig aus,2022-09-28,petrakP8081RR,NaN,Da ich mein Menü zuerst bekam ahnten wir es no...,4.0,l'osteria,de,de,Top Preis/Leistung und auch der Geschmack stim...,3
2996,2996,902237029,7335669,0,Sehr gut,2023-07-11,Carsten E,NaN,Also ich fand den Service und das Essen Top.,4.0,l'osteria,de,de,Ich war nur auf der Durchreise in Osnabrück un...,3
2997,2997,881299277,3642346,0,Riesige Pizzen,2023-03-07,895anikag,NaN,Wir kommen gerne wieder.,4.0,l'osteria,de,de,Wir wurden sehr freundlich empfangen und musst...,8
2998,2998,892594569,6558079,0,In diesem Jahr war es leider nicht so gut.,2023-05-30,inesbaer,"Friedberg, Deutschland","Leider war meine Lasagne sehr trocken, ich hab...",4.0,l'osteria,de,de,"Wir haben den Eindruck, dass die Qualität leid...",1


In [14]:
rating_counts = df['rating'].value_counts()
rating_counts

rating
3.0    600
4.0    600
5.0    600
2.0    600
1.0    600
Name: count, dtype: int64

In [15]:
rating_counts = df['restaurant_name'].value_counts()
rating_counts

restaurant_name
l'osteria        1479
hans im glück     548
block house       447
vapiano           416
dean&david        110
Name: count, dtype: int64

In [16]:
df.groupby(['sentence_idx', 'review_id']).size().reset_index(name='count')

,sentence_idx,review_id,count
0,0,845622217,1
1,0,845764015,1
2,0,846016475,1
3,0,846184126,1
4,0,846505990,1
...,...,...,...
2995,32,894822345,1
2996,33,894822345,1
2997,35,846184126,1
2998,35,894822345,1
